In [1]:
import pandas as pd
import math

In [2]:
def dotDist(p1, p2, p3):
    # Finds the minimal distance between a point p3 and
    # the line segment connecting points p1 and p2
    if len(p1)==len(p2) and len(p2)==len(p3):
        p31 = [a-b for (a,b) in zip(p3,p1)]
        p21 = [a-b for (a,b) in zip(p2,p1)]
        t = sum([a*b for (a,b) in zip(p31, p21)])/sum([a**2 for a in p21])
        t = min(1,max(t,0))
        p0 = [a+t*b for (a,b) in zip(p1,p21)]
        dist = math.dist(p3,p0)
        return dist
    else:
        raise ValueError("dimensions of the three points do not match")

In [3]:
def xangle(p1, p2):
    # Computes the angle between the vector from p1 to p2, and the X-axis
    d = math.dist(p1,p2)
    if d>0:
        theta = (p2[0]-p1[0])/d
        theta = math.acos(theta)
        theta = theta+2*(math.pi-theta)*(p2[1]<p1[1])
    else:
        theta =0
    return theta

In [4]:
class Polygons:
    # inputs are coordinates (x,y) of all vertices of a convex polygon in any order
    # x: x-coordinate of vertices ordered counter-clockwise, starting with the vertex with the smallest y-coordinate.
    # y: y-coordinate of vertices ordered counter-clockwise, starting with the vertex with the smallest y-coordinate.
    # in case of tie, pick the one with smallest x
    
    # ang: the polar angle of each edge
    
    def __init__(self, *args):
        # find the starting point p1
        df = pd.DataFrame(args, columns=['x','y'])
        df.drop_duplicates(inplace = True)
        df = df.sort_values(by=['y','x'])
        p1 = df.iloc[0,:]
        n = df.shape[0]
        
        # calculate theta between p1 and other vertices
        theta = []
        for i in range(n):
            theta.append(xangle(p1, df.iloc[i,:]))
        df['theta'] = theta
        
        # sort by theta (counter-clockwise order)
        df.sort_values(by=['theta'], inplace = True)
        
        # get rid of non-vertex points:
        # caculate d, the distance between any vertex to the line segment connecting the previous and next vertices
        # if d is 0, drop the middle point
        d = [dotDist(df.iloc[-1,0:2], df.iloc[1,0:2], df.iloc[0,0:2])]
        for i in range(1,n-1):
            d.append(dotDist(df.iloc[i-1,0:2], df.iloc[i+1,0:2], df.iloc[i,0:2]))
        d.append(dotDist(df.iloc[-2,0:2], df.iloc[0,0:2], df.iloc[-1,0:2]))
        df['d'] = d
        df = df.loc[df['d']>0.0]
        
        # caculate the polar angle of each edge i (from vertex i to vertex i+1)
        n = df.shape[0]
        angle = [None]*n
        for i in range(n-1):
            angle[i] = (xangle(df.iloc[i,0:2], df.iloc[i+1,0:2]))
        angle[-1] = (xangle(df.iloc[-1,0:2], df.iloc[0,0:2]))
        df['angle'] = angle
        
        self.x = list(df['x'])
        self.y = list(df['y'])
        self.ang = list(df['angle'])
        self.dim = n

In [5]:
def minksum(P1:Polygons, P2:Polygons):
    m = P1.dim
    n = P2.dim
    i = 0
    j = 0
    P1.x+=[P1.x[0]]
    P2.x+=[P2.x[0]]
    P1.y+=[P1.y[0]]
    P2.y+=[P2.y[0]]
    P1.ang+=[P1.ang[0]]
    P2.ang+=[P2.ang[0]]
    P = ()
    while i<m or j<n:
        P+=([P1.x[i]+P2.x[j],P1.y[i]+P2.y[j]],)
        if i == m:
            j+=1
        elif j==n:
            i+=1
        else:        
            dif = P1.ang[i]-P2.ang[j]
            if dif>=0:
                j+=1
            if dif<=0:
                i+=1
    return Polygons(*P)

# Testing 

In [6]:
P1 = Polygons([0.0,0.4],[-1.1,0.0],[1.1,0.0],[0.0,-0.6])
P2 = Polygons([-1.0,0.0],[1.0,0.1],[-1.0,0.55],[1.1,0.5])
print(P1.ang)
print(P2.ang)

[0.4993467216801301, 2.792821650005886, 3.4903636571737002, 5.783838585499456]
[0.049958395721943306, 1.325817663668032, 3.117787627404723, 4.71238898038469]


In [7]:
P = minksum(P1, P2)
print(P.x)
print(P.y)
P.dim

[-1.0, 1.0, 2.1, 2.2, 1.1, -1.0, -2.1, -2.1]
[-0.6, -0.5, 0.1, 0.5, 0.9, 0.9500000000000001, 0.55, 0.0]


8

## If I don't force one index to increase when another index has reached the boundary:

In [8]:
P1 = Polygons([0.0,0.4],[-1.1,0.0],[1.1,0.0],[0.0,-0.6])
P2 = Polygons([-1.0,0.0],[1.0,0.1],[-1.0,0.55],[1.1,0.5])

m = P1.dim
n = P2.dim
i = 0
j = 0
P1.x+=[P1.x[0]]
P2.x+=[P2.x[0]]
P1.y+=[P1.y[0]]
P2.y+=[P2.y[0]]
P1.ang+=[P1.ang[0]]
P2.ang+=[P2.ang[0]]
P = ()
while i<m or j<n:
    P+=([P1.x[i]+P2.x[j],P1.y[i]+P2.y[j]],)        
    dif = P1.ang[i]-P2.ang[j]
    if dif>=0:
        j+=1
    if dif<=0:
        i+=1

IndexError: list index out of range

In [9]:
(i, j) # corresponds to (4,6) in Julia

(3, 5)